# <div style="color:white;display:fill;border-radius:5px;background-color:#0E2031;letter-spacing:0.5px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Conteúdo</p></div>     
   
- Libraries
- Configurations
- Loading Data
- Glossary
- **Data Preprocessing** 
    - Missing Data
    - Duplicated Values
    - Data Types
    - Strange Values
    - Feature Engineer

## <div style="color:white;display:fill;border-radius:5px;background-color:#153656;letter-spacing:0.5px;overflow:hidden"><p style="padding:15px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Libraries</p></div>

In [11]:
#!pip install -Uq pandas
#!pip install -q watermark
#!pip install pycountry-convert
#!pip install python-dotenv

In [12]:
# Basic Tools
import numpy as np
import pandas as pd
from datetime import datetime as dt

# Convert Alpha3 Countries to Country Name and Continents
import pycountry
import pycountry_convert as pc


# File/ OS Tools
import os
import sys
import dotenv
from watermark import watermark

## <div style="color:white;display:fill;border-radius:5px;background-color:#153656;letter-spacing:0.5px;overflow:hidden"><p style="padding:15px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Configurations</p></div>

In [13]:
print(watermark())

Last updated: 2023-05-29T18:24:12.261555-03:00

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.5.0

Compiler    : GCC 11.3.0
OS          : Linux
Release     : 5.19.0-42-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 12
Architecture: 64bit



In [14]:
# dotenv.load_dotenv(dotenv.find_dotenv())
# ROOT_DIR = os.getenv('ROOT_DIR')

ROOT_DIR = '/home/alysson/projects/Hotel-Booking-Cancelations'

In [7]:
if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)

os.chdir(ROOT_DIR)

## <div style="color:white;display:fill;border-radius:5px;background-color:#153656;letter-spacing:0.5px;overflow:hidden"><p style="padding:15px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Loading Data</p></div>

In [9]:
for dirname, _, filenames in os.walk(os.path.join(ROOT_DIR, "data")):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/home/alysson/projects/Hotel-Booking-Cancelations/data/data_processed/.gitignore
/home/alysson/projects/Hotel-Booking-Cancelations/data/data_processed/data.csv
/home/alysson/projects/Hotel-Booking-Cancelations/data/data_processed/hotel_bookings_processed.csv
/home/alysson/projects/Hotel-Booking-Cancelations/data/data_processed/data.csv.dvc
/home/alysson/projects/Hotel-Booking-Cancelations/data/data_raw/hotel_bookings.csv


In [10]:
DATA_RAW_PATH = os.path.join(ROOT_DIR, "data", "data_raw", "hotel_bookings.csv")

In [77]:
data_raw = pd.read_csv(DATA_RAW_PATH)

In [78]:
data = data_raw.copy()

In [79]:
data.shape

(119390, 32)

## <div style="color:white;display:fill;border-radius:5px;background-color:#153656;letter-spacing:0.5px;overflow:hidden"><p style="padding:15px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Glossary</p></div>

In [80]:
glossary_pt = [['Colunas', 'Significado'],
              ['hotel', 'Hotel Urbano ou Resort'],
              ['is_canceled', 'Informa se a reserva foi cancelada (1 = Cancelado, 0 = Não Cancelado)'],
              ['lead_time', 'Número de dias entre a data da reserva até a data do check-in'],
              ['arrival_date_year', 'Ano da data de chegada'],
              ['arrival_date_month', 'Mês da data de chegada'],
              ['arrival_date_week_numeber', 'Semana do ano da data de chegada'],
              ['arrival_date_month', 'Mês da data de chegada'],
              ]
#print(tabulate(glossary_pt, headers='firstrow', stralign='left', tablefmt='simple'))

## <div style="color:white;display:fill;border-radius:5px;background-color:#153656;letter-spacing:0.5px;overflow:hidden"><p style="padding:15px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Data Preprocessing</p></div>

### <div style="color:white;display:fill;border-radius:5px;background-color:#2E6B8E;letter-spacing:0.5px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Missing Data</p></div>

In [81]:
data.isna().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [82]:
nan_replacements = {"children": 0, "agent": 0, "company": 0, "country":"Unknow"}
data = data.fillna(nan_replacements)

**Observações:**

- Are the NaN values because the customer did not inform the nationality when making the reservation or the system was not able to identify the user's nationality?

- Why so many missing values in the Company column?

### <div style="color:white;display:fill;border-radius:5px;background-color:#2E6B8E;letter-spacing:0.5px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Duplicated Data</p></div>

In [83]:
data.duplicated().sum()

31994

Duplicate values here are not necessarily a problem. It is possible for example to have reservations for the same type of room on the same day that were booked on the same day. Since we don't have more information if these values refer to the same guest, we'll keep it as it is.

In [84]:
data.loc[data.duplicated(), :].sample(3)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
43765,City Hotel,0,86,2015,September,39,24,0,2,2,0.0000,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,12.0000,0.0000,0,Transient-Party,105.0000,0,0,Check-Out,2015-09-26
70858,City Hotel,1,233,2017,June,25,23,1,2,2,0.0000,0,BB,PRT,Groups,TA/TO,0,0,0,A,A,0,Non Refund,20.0000,0.0000,0,Transient,105.0000,0,0,Canceled,2017-04-21
32228,Resort Hotel,0,40,2017,January,3,16,1,2,1,0.0000,0,Undefined,PRT,Groups,Direct,0,0,0,A,E,1,No Deposit,0.0000,0.0000,0,Transient-Party,55.0000,0,0,Check-Out,2017-01-19


### <div style="color:white;display:fill;border-radius:5px;background-color:#2E6B8E;letter-spacing:0.5px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Data Types</p></div>

In [85]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119390 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [86]:
data['reservation_status_date'] = pd.to_datetime(data['reservation_status_date'])#, format='%Y/%m/%d'

In [87]:
# Dicionário de mapeamento dos meses
# meses = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}
# data['arrival_date_month'] = data['arrival_date_month'].map(meses)

In [88]:
data.head(1)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0000,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,0.0000,0.0000,0,Transient,0.0000,0,0,Check-Out,2015-07-01


In [89]:
# data.rename(columns = {'arrival_date_year':'year','arrival_date_month':'month','arrival_date_day_of_month':'day'}, inplace=True)
# data['arrival_date'] = pd.to_datetime(data[['year', 'month', 'day']],format='%Y%m%d')
# data=data.drop(columns=['year','month','day'], axis=1)

In [90]:
#data['time']= (data.arrival_date - data.reservation_status_date).dt.days

In [91]:
data = data.astype({"children": int, "agent": int, "company": int})

### <div style="color:white;display:fill;border-radius:5px;background-color:#2E6B8E;letter-spacing:0.5px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Dados Estranhos</p></div>

In [92]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
is_canceled,"119,390.0000",0.3704,0.4829,0.0000,0.0000,0.0000,1.0000,1.0000
lead_time,"119,390.0000",104.0114,106.8631,0.0000,18.0000,69.0000,160.0000,737.0000
arrival_date_year,"119,390.0000","2,016.1566",0.7075,"2,015.0000","2,016.0000","2,016.0000","2,017.0000","2,017.0000"
arrival_date_week_number,"119,390.0000",27.1652,13.6051,1.0000,16.0000,28.0000,38.0000,53.0000
arrival_date_day_of_month,"119,390.0000",15.7982,8.7808,1.0000,8.0000,16.0000,23.0000,31.0000
stays_in_weekend_nights,"119,390.0000",0.9276,0.9986,0.0000,0.0000,1.0000,2.0000,19.0000
stays_in_week_nights,"119,390.0000",2.5003,1.9083,0.0000,1.0000,2.0000,3.0000,50.0000
adults,"119,390.0000",1.8564,0.5793,0.0000,2.0000,2.0000,2.0000,55.0000
children,"119,390.0000",0.1039,0.3986,0.0000,0.0000,0.0000,0.0000,10.0000
babies,"119,390.0000",0.0079,0.0974,0.0000,0.0000,0.0000,0.0000,10.0000


In [93]:
#No people and reservation status equal to "check-out"
len(data.loc[((data['children'] == 0) & (data['adults'] == 0) & (data['babies'] == 0) & (data['reservation_status'] == 'Check-Out'))])

155

In [94]:
data=data.loc[~((data['children'] == 0) & (data['adults'] == 0) & (data['babies'] == 0) & (data['reservation_status'] == 'Check-Out'))]

In [95]:
#Negative ADR 
data = data.loc[~(data['adr']<0)]
len(data)

119234

### <div style="color:white;display:fill;border-radius:5px;background-color:#2E6B8E;letter-spacing:0.5px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:120%">Feature Engineer</p></div>

In [96]:
#Same meaning values according to dictionary source
data['meal'] = data['meal'].replace("Undefined", "SC")

In [97]:
#data["adr_pp"] = data["adr"] / (data["adults"] + data["children"])
data["people"] = (data["adults"] + data["children"] + data["babies"])
data['kids'] = data['children'] + data['babies']
data['days_stay'] = data['stays_in_weekend_nights'] + data['stays_in_week_nights']

In [98]:
def add_country_names(df):
    country_names = []
    for country_code in df['country']:
        try:
            country_name = pycountry.countries.get(alpha_3=country_code).name
        except AttributeError:
            country_name = None
        except LookupError:
            country_name = None
        country_names.append(country_name)
    df['country_name'] = country_names
    return df

In [99]:
def add_continent(df):
    continents = []
    for country in df['country_name']:
        try:
            country_code = pc.country_name_to_country_alpha2(country)
            continent_name = pc.country_alpha2_to_continent_code(country_code)
            continent_code = pc.convert_continent_code_to_continent_name(continent_name)
            continents.append(continent_code)
        except:
            continents.append(None)
    df['continentes'] = continents    

    return df

In [100]:
data = add_country_names(data)

In [101]:
data = add_continent(data)

In [102]:
data.loc[data['country'] == 'PRT', 'continentes'] = 'Native'

In [103]:
nan_replacements = {"children": 0, "agent": 0, "company": 0, "country":"Unknow", "country_name":"Unknow", "continentes":"Unknow"}
data = data.fillna(nan_replacements)

In [104]:
data.isna().sum()

hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
agent                             0
company                           0
days_in_waiting_list              0
customer_type                     0
adr                         

In [105]:
data=data.reset_index(drop=True)

In [106]:
data.to_csv(f"{ROOT_DIR}/data/data_processed/hotel_bookings_processed.csv", index=False)